# Rap Music Analysis

In [6]:
# ! pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
# Using Chrome to acess web
driver = webdriver.Chrome(ChromeDriverManager().install())

# Open the website
driver.get("https://genius.com/#top-songs")

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/sarahamiraslani/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


There are a number of ways to tell our webdriver what elements to find, all of which use selectors. A selector is a unique idetnfier for an element on a webpage. To find the selector for a specific element, we need to inspect the webpage. Doing so wil show use the HTML underlying any webpage. 

Xpath is the language used for locating nodes in an XML document. One of the main reasons for using Xpath is when you don't have a suitable id or name attribute for the element you wish to locate. You can use XPath to either locate the element in absolute terms (not advised), or relative to an element that does have an id or name attribute. XPath locators can also be used to specify elements via attributes other than id and name.

In [21]:
# Tell webdriver to select Genre as rap
rap_button = driver.find_elements_by_xpath('//*[@id="top-songs"]/div/div[2]/div/div/div[2]/div[2]/div[3]/div')[0]
rap_button.click()


# Scrape songs






# #top-songs > div > div.PageGridCenter-q0ues6-0.Charts__LoadMore-sc-1re0f44-1.eDwRUT > div
# load_more = driver.find_elements_by_xpath('/html/body/div/div/div[5]/div[2]/div/div[4]')
# load_more.click()
# #top-songs > div > div.PageGridCenter-q0ues6-0.Charts__LoadMore-sc-1re0f44-1.eDwRUT > div

In [30]:
songs = driver.find_elements_by_xpath('//*[@id="top-songs"]/div/div[3]')[0]






# for song in songs:
#     print(song)

# # # songs_list = []

# for song in range(len(songs)):
#     songs_list.append(songs[song].text)
    
# songs_list = songs_list[0]

# songs_list.split('\n')

TypeError: 'WebElement' object is not iterable

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create an empty DataFrame
column_names = ['Title','Artist','Lyrics','Comments']
df = pd.DataFrame(columns = column_names)

In [ ]:
import getpass
import requests

def request_artist_info(artist_name, page):
    
    base_url = 'https://api.genius.com'
    token = getpass.getpass('Enter your Genius API token:')
    
    headers = {'Authorization': 'Bearer ' + token}
    search_url = base_url + '/search?per_page=50&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    
    return response 

In [ ]:
import json

response = request_artist_info('Drake', 1)
json = response.json()

In [ ]:
artist = []
song_name = []
pageviews = []

for hit in json['response']['hits']: 
    artist.append(hit['result']['primary_artist']['name'])
    song_name.append(hit['result']['title'])
    pageviews.append(hit['result']['stats']['pageviews'])

In [ ]:
df['Title']= song_name
df['Artist'] = artist
df['pageviews'] = pageviews

In [ ]:
def request_song_url(artist_name, song_cap):

    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()       
        song_info = []
        
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
    
    return songs

In [ ]:
urls = request_song_url('drake',20)

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

counter = 0

for url in urls: 
    
    req = Request(url, headers = {"User-Agent" : "Mozilla/5.0"})
    webpage = urlopen(req).read()

    # Create a BeautifulSoup object 
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    songs = {}
    songs['lyrics'] = [];
    songs['comments'] = [];

    # Extract user comments on the song
    for div in soup.findAll('div',attrs={'class':'rich_text_formatting'}):
        comments = div.text.strip().split("\n")
        
        for comment in comments: 
            if comment != "":
                songs['comments'].append(comment);
                df['Comments'].iloc[counter] = songs['comments']
        
    # Extract the lyrics of the song
    for div in soup.findAll('div', attrs = {'class':'lyrics'}):
        songs['lyrics'].append(div.text.strip().split("\n"));
        df['Lyrics'].iloc[counter] = songs['lyrics'][0]
    
    counter +=1

In [ ]:
df.head()

In [ ]:
import string

def clean_lyrics(lst):
    
    new_lst = []
    for i in lst:
        if '['in i: 
            continue
        else: 
            new_lst.append(i)
    
    my_string = ' '.join(new_lst)
    new_str = my_string.translate(str.maketrans('','', string.punctuation))
    return new_str

In [ ]:
df['Lyrics'] = df['Lyrics'].apply(clean_lyrics)

In [ ]:
def clean_comments(lst):
    my_string = ' '.join(lst)
    new_str = my_string.translate(str.maketrans('', '', string.punctuation))
    return new_str

In [ ]:
df['Comments'] = df['Comments'].apply(clean_comments)

In [ ]:
df.head()

In [ ]:
import spacy

# Load the language library
nlp = spacy.load('en')

def get_lemmas(string):

    # Create a document object
    doc = nlp(string)

    lemmas = []

    for token in doc: 
        if token.pos == 103: # takes care of spaces
            continue
        elif token.lemma_ == '-PRON-':
            lemmas.append(token)
        else: 
            lemmas.append(token.lemma_)
            
    return lemmas

In [ ]:
df['lyric_lemmas'] = df['Lyrics'].apply(get_lemmas)

In [ ]:
df.head()

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()
df['comments_scores'] = df['Comments'].apply(lambda comment: sid.polarity_scores(comment))
df['lyrics_scores'] = df['Lyrics'].apply(lambda lyric: sid.polarity_scores(lyric))

In [ ]:
df['comments_compound'] = df['comments_scores'].apply(lambda d:d['compound'])
df['lyrics_compound'] = df['lyrics_scores'].apply(lambda d:d['compound'])

In [ ]:
df.head()

In [ ]:
plt.hist(df['lyrics_compound'])

In [ ]:
plt.bar(df['Title'],df['pageviews'])
plt.ylabel('Page Views')
plt.title('Genius Page Views by Song')
plt.xticks(rotation=45, ha="right")
plt.show()